
---

# 0. 実行環境

* **Language/Runtime:** JavaScript (Node.js v22.14.0)
* **Module:** CommonJS
* **外部ライブラリ:** 使用不可（Node 標準のみ）
* **CI 前提:** `node solution.js` 実行可能（I/O なし／関数のみ）

---

# 1. 問題の分析

## 競技プログラミング視点

* 4 セグメント（各 1〜3 桁、0〜255、先頭 0 禁止）という**強い制約**があるため、深さ最大 4 の探索で完全探索しても軽い。
* 文字列長は有効 IP では **4〜12** に限定される（これ以外は即時空配列）。
* 各セグメントは長さ候補が 1〜3 の **最大 3 分岐**。よって理論上の探索ノードは高々 (3^4=81) 程度＋枝刈りでさらに減る。

## 業務開発視点

* **保守性**：`isValidSegment` のような小関数に分けず、ホットパスのクロージャ生成を抑えるため **1 関数内で判定を完結**。
* **例外方針**：問題前提外（非文字列・非数字）には `TypeError` を投げる。範囲外長は業務上は空配列返す方が扱いやすい（例外ではない）。

## JavaScript特有の考慮点

* **V8 最適化**：数値化は `charCodeAt` を用いた逐次加算で `parseInt` を回避し、**分岐ごとに最大 3 回**の軽量演算に抑制。
* **GC 対策**：結果バッファ `path` を**固定長配列(長さ4)** で再利用。部分文字列は `len===1` のときは `s[idx]` を使用して短命オブジェクトを最小化。
* **ループ**：`for` ベースで単純化、スプレッド/高階関数は使用しない。

---

# 2. アルゴリズムアプローチ比較

| アプローチ               | 時間計算量                                |              空間計算量 | JS実装コスト | 可読性 | 備考                    |
| ------------------- | ------------------------------------ | -----------------: | ------: | --: | --------------------- |
| 方法A: DFS+枝刈り（本採用）   | 高々 (3^4) ≒ **O(1)**（入力長で見ても O(n) 未満） | **O(1)**（深さ4・固定配列） |       低 |   中 | 長さ制約と 0/255 判定で強力に枝刈り |
| 方法B: 3ドット全探索（i<j<k） | 最悪 **O(n³)**（ただし n≤12）               |               O(1) |     低〜中 |   中 | 実装は簡単だが分岐判定が散在        |
| 方法C: BFS（幅優先）       | 節点数は DFS 同等で実質 **O(1)**              |             O(候補数) |       低 |   高 | 階層ごとだが中間配列が増える        |

---

# 3. 選択したアルゴリズムと理由

* **選択:** 方法A（DFS + 強い枝刈り）
* **理由:**

  * 深さ固定（4）・分岐最大 3 で**理論上の上限が極小**。最速かつコード量も少ない。
  * 必要十分な**枝刈り（残文字数チェック、先頭 0、>255）**で無駄探索をほぼ排除。
* **JavaScript特有の最適化ポイント:**

  * `charCodeAt` による**逐次数値化**（`val = val*10 + digit`）。
  * `path` は**固定長 4** を再利用、都度 new しない。
  * 早期長チェック（`n<4 || n>12`）で**即 return**。

---

# 4. コード実装（solution.js） — *LeetCode 形式（Pure Function／I/Oなし）*

```javascript
'use strict';

/**
 * Restore IP Addresses (LeetCode)
 * 純関数・I/Oなし。入力は数字文字列のみを想定。
 *
 * @param {string} s - 数字のみから成る文字列
 * @returns {string[]} 生成可能な全ての有効なIPv4文字列（順不同）
 * @throws {TypeError} 引数が文字列でない／数字以外を含む場合
 * @complexity
 *  時間: 高々 3^4 分岐 + 軽量チェックで実質 O(1)（n<=12 かつ各分岐で最大3桁のみ）
 *  空間: O(1) 追加（深さ4の固定配列 path のみ、結果配列は出力）
 */
var restoreIpAddresses = function (s) {
  // ---- 入力検証（軽量 & 早期）----
  if (typeof s !== 'string') throw new TypeError('Input must be a string.');
  // 問題前提外にも耐える：数字のみ許可
  // （LeetCode想定では必ず数字だが、業務コードとして堅牢化）
  for (let i = 0, n = s.length; i < n; i++) {
    const c = s.charCodeAt(i);
    if (c < 48 || c > 57) throw new TypeError('Input must contain digits only.');
  }

  const n = s.length;
  // 有効なIPv4の総桁数は 4〜12（各セグメント1〜3桁）
  if (n < 4 || n > 12) return [];

  const res = [];
  const path = new Array(4); // セグメント4つを固定長で再利用

  /**
   * DFS（idx: 次に読む位置, seg: 何個目のセグメントか）
   * 残文字数と残セグメント数で強い枝刈りを行う。
   */
  function dfs(idx, seg) {
    if (seg === 4) {
      if (idx === n) res.push(path.join('.'));
      return;
    }

    const remainSegs = 4 - seg;
    const remainChars = n - idx;
    // 残文字が足りない／多すぎる場合は即棄却
    if (remainChars < remainSegs || remainChars > remainSegs * 3) return;

    let val = 0;

    // len: 現セグメントの長さ（1〜3）
    // 先頭が '0' の場合は "0" のみ許可（leading zero 禁止）
    const firstIsZero = s.charCodeAt(idx) === 48;
    const maxLen = firstIsZero ? 1 : 3;

    for (let len = 1; len <= maxLen && idx + len <= n; len++) {
      // 数値を逐次形成（parseInt回避）
      val = val * 10 + (s.charCodeAt(idx + len - 1) - 48);
      if (val > 255) break; // 255 超過で以降の長い桁も全て超過

      // 文字列化は必要最小限（len===1 は s[idx] をそのまま）
      path[seg] = (len === 1) ? s[idx] : s.slice(idx, idx + len);
      dfs(idx + len, seg + 1);
    }
  }

  dfs(0, 0);
  return res;
};

// Node実行/テスト環境用（LeetCodeでは無視されても害はない）
if (typeof module !== 'undefined') {
  module.exports = restoreIpAddresses;
}

Analyze Complexity
Runtime 0 ms
Beats 100.00%

Memory 54.24 MB
Beats 91.44%

```

---

# 5. 追加メモ（JS最適化チェックリスト）

* ループは `for` を基本、`map/forEach` 不使用で**割り当て削減**。
* `path` は**固定長再利用**、`Array.join('.')` は最後にのみ実行。
* 先頭ゼロ・範囲（0〜255）・残文字数は**早期枝刈り**で探索爆発を防止。
* 数値化は `charCodeAt` を使い**逐次演算**で `parseInt` 回避。
* 例外は**ホットパス外（入口）**で早期に弾く。

---

必要に応じて、同アプローチの **3 ドット全探索版（i<j<k の三重ループ）** も提示できますが、上記 DFS 実装が最速・最小メモリで安定です。
